# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2: Web Scraping with Beautiful Soup

**Harvard University**<br>
**Fall 2019**<br>
**Instructors:** Pavlos Protopapas, Kevin Rader, and Chris Tanner <br>
**Lab Instructors:** Chris Tanner and Eleni Kaxiras<br>

**Authors:** Rahul Dave, David Sondak, Will Claybaugh, Pavlos Protopapas, Chris Tanner, and Eleni Kaxiras

---

In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
import requests
def css_styling():
    url = "https://raw.githubusercontent.com/santiagoa88g/2019-CS109A/master/content/styles/cs109.css"
    response = requests.get(url)
    styles = response.text
    return HTML(styles)
css_styling()

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Introduction to Web Servers and HTTP </li>
<li> Download webpages and get basic properties </li>
<li> Parse the page with Beautiful Soup</li>
<li> String formatting</li>
<li> Additonal Python/Homework Comment</li>
<li> Walkthrough Example</li>
</ol>

# Learning Goals

- Understand the structure of a web page
- Understand how to use Beautiful soup to scrape content from web pages.
- Feel comfortable storing and manipulating the content in various formats.
- Understand how to convert structured format into a Pandas DataFrame

In this lab, we'll scrape Goodread's Best Books list:

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1 .

We'll walk through scraping the list pages for the book names/urls. First, we start with an even simpler example.

*This lab corresponds to lectures #2 and #3 and maps on to Homework #1 and further.*

# 1. Introduction to Web Servers and HTTP

A web server is just a computer -- usually a powerful one, but ultimately it's just another computer -- that runs a long/continuous process that listens for requests on a pre-specified (Internet) _port_ on your computer. It responds to those requests via a protocol called HTTP (HyperText Transfer Protocol). HTTPS is the secure version. When we use a web browser and navigate to a web page, our browser is actually sending a request on our behalf to a specific web server. The browser request is essentially saying "hey, please give me the web page contents", and it's up to the browser to correctly render that raw content into a coherent manner, dependent on the format of the file. For example, HTML is one format, XML is another format, and so on.

Ideally (and usually), the web server complies with the request and all is fine. As part of this communication exchange with web servers, the server also sends a status code.
- If the code starts with a **2**, it means the request was successful.
- If the code starts with a **4**, it means there was a client error (you, as the user, are the client). For example, ever receive a 404 File Not Found error because a web page doesn't exist? This is an example of a client error, because you are requesting a bogus item.
- If the code starts with a **5**, it means there was a server error (often that your request was incorrectly formed).

[Click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

As an analogy, you can think of a web server as being like a server at a restaurant; its goal is _serve_ you your requests. When you try to order something not on the menu (i.e., ask for a web page at a wrong location), the server says 'sorry, we don't have that' (i.e., 404, client error; your mistake).

**IMPORTANT:**
As humans, we visit pages in a sane, reasonable rate. However, as we start to scrape web pages with our computers, we will be sending requests with our code, and thus, we can make requests at an incredible rate. This is potentially dangerous because it's akin to going to a restaurant and bombarding the server(s) with thousands of food orders. Very often, the restaurant will ban you (i.e., Harvard's network gets banned from the website, and you are potentially held responsible in some capacity?). It is imperative to be responsible and careful. In fact, this act of flooding web pages with requests is the single-most popular, yet archiac, method for maliciously attacking websites / computers with Internet connections. In short, be respectful and careful with your decisions and code. It is better to err on the side of caution, which includes using the **``time.sleep()`` function** to pause your code's execution between subsequent requests. ``time.sleep(2)`` should be fine when making just a few dozen requests. Each site has its own rules, which are often visible via their site's ``robots.txt`` file.

### Additional Resources

**HTML:** if you are not familiar with HTML see https://www.w3schools.com/html/ or one of the many tutorials on the internet.

**Document Object Model (DOM):** for more on this programming interface for HTML and XML documents see https://www.w3schools.com/js/js_htmldom.asp.

# 2. Download webpages and get basic properties

``Requests`` is a highly useful Python library that allows us to fetch web pages.
``BeautifulSoup`` is a phenomenal Python library that allows us to easily parse web content and perform basic extraction.

If one wishes to scrape webpages, one usually uses ``requests`` to fetch the page and ``BeautifulSoup`` to parse the page's meaningful components. Webpages can be messy, despite having a structured format, which is why BeautifulSoup is so handy.

Let's get started:

In [3]:
from bs4 import BeautifulSoup
import requests

To fetch a webpage's content, we can simply use the ``get()`` function within the requests library:

In [4]:
url = "https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url) # you can use any URL that you wish

The response variable has many highly useful attributes, such as:
- status_code
- text
- content

Let's try each of them!

### response.status_code

In [5]:
response.status_code

200

You should have received a status code of 200, which means the page was successfully found on the server and sent to receiver (aka client/user/you). [Again, you can click here](https://www.restapitutorial.com/httpstatuscodes.html) for a full list of status codes.

### response.text


In [6]:
response.text

'<!doctype html><html class="no-js" lang="en"><head><!-- OneTrust Cookies Consent Notice start for npr.org -->\n<script type="text/javascript" src="https://cdn.cookielaw.org/consent/82089dfe-410c-4e1b-a7f9-698174b62a86/OtAutoBlock.js" ></script>\n<script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js"  type="text/javascript" charset="UTF-8" data-domain-script="82089dfe-410c-4e1b-a7f9-698174b62a86" ></script>\n\n<script type="text/javascript">\nfunction OptanonWrapper() {\n    NPR_OptanonWrapper = true;\n    document.dispatchEvent(new CustomEvent(\'npr:DataConsentAvailable\'));\n    \n    OneTrust.OnConsentChanged(function() {\n        document.dispatchEvent(new CustomEvent(\'npr:DataConsentChanged\'));\n    });\n }\n</script>\n<!-- OneTrust Cookies Consent Notice end for npr.org -->\n<script src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript" charset="UTF-8" ccpa-opt-out-ids="C0004" ccpa-opt-out-geo="us" ccpa-opt-out-lspa="false"></script><script ty

Holy moly! That looks awful. If we use our browser to visit the URL, then right-click the page and click 'View Page Source', we see that it is identical to this chunk of glorious text.

### response.content

In [7]:
response.content

b'<!doctype html><html class="no-js" lang="en"><head><!-- OneTrust Cookies Consent Notice start for npr.org -->\n<script type="text/javascript" src="https://cdn.cookielaw.org/consent/82089dfe-410c-4e1b-a7f9-698174b62a86/OtAutoBlock.js" ></script>\n<script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js"  type="text/javascript" charset="UTF-8" data-domain-script="82089dfe-410c-4e1b-a7f9-698174b62a86" ></script>\n\n<script type="text/javascript">\nfunction OptanonWrapper() {\n    NPR_OptanonWrapper = true;\n    document.dispatchEvent(new CustomEvent(\'npr:DataConsentAvailable\'));\n    \n    OneTrust.OnConsentChanged(function() {\n        document.dispatchEvent(new CustomEvent(\'npr:DataConsentChanged\'));\n    });\n }\n</script>\n<!-- OneTrust Cookies Consent Notice end for npr.org -->\n<script src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript" charset="UTF-8" ccpa-opt-out-ids="C0004" ccpa-opt-out-geo="us" ccpa-opt-out-lspa="false"></script><script t

What?! This seems identical to the ``.text`` field. However, the careful eye would notice that the very 1st characters differ; that is, ``.content`` has a *b'* character at the beginning, which in Python syntax denotes that the data type is bytes, whereas the ``.text`` field did not have it and is a regular String.

Ok, so that's great, but how do we make sense of this text? We could manually parse it, but that's tedious and difficult. As mentioned, BeautifulSoup is specifically designed to parse this exact content (any webpage content).

## BEAUTIFUL SOUP
![title](images/soup_for_you.jpg) (property of NBC)


The [documentation for BeautifulSoup is found here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

A BeautifulSoup object can be initialized with the ``.content`` from request and a flag denoting the type of parser that we should use. For example, we could specify ``html.parser``, ``lxml``, etc [documentation here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#differences-between-parsers). Since we are interested in standard webpages that use HTML, let's specify the html.parser:

In [8]:
soup = BeautifulSoup(response.content, "html.parser")
soup

<!DOCTYPE html>
<html class="no-js" lang="en"><head><!-- OneTrust Cookies Consent Notice start for npr.org -->
<script src="https://cdn.cookielaw.org/consent/82089dfe-410c-4e1b-a7f9-698174b62a86/OtAutoBlock.js" type="text/javascript"></script>
<script charset="UTF-8" data-domain-script="82089dfe-410c-4e1b-a7f9-698174b62a86" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript"></script>
<script type="text/javascript">
function OptanonWrapper() {
    NPR_OptanonWrapper = true;
    document.dispatchEvent(new CustomEvent('npr:DataConsentAvailable'));
    
    OneTrust.OnConsentChanged(function() {
        document.dispatchEvent(new CustomEvent('npr:DataConsentChanged'));
    });
 }
</script>
<!-- OneTrust Cookies Consent Notice end for npr.org -->
<script ccpa-opt-out-geo="us" ccpa-opt-out-ids="C0004" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript"></script><script class="optanon-category-C

Alright! That looks a little better; there's some whitespace formatting, adding some structure to our content! HTML code is structured by `<tags>`. Every tag has an opening and closing portion, denoted by ``< >`` and ``</ >``, respectively. If we want just the text (not the tags), we can use:

In [9]:
soup.get_text()

"\n\n\n\n\n\n\n\n\n\n\n\nWhat If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAccessibility links \nSkip to main content\nKeyboard shortcuts for audio player\n\n\n\n\n\n\n\n\n                    Open Navigation Menu\n                \n\n\n\n\n\n\n\n\n\nNPR Shop\n\n\n\n\n\n\n\n>\n                    Close Navigation Menu\n\n\n\n\nHome\n\n\n\nNews\nExpand/collapse submenu for News\n\n\nNational\nWorld\nPolitics\nBusiness\nHealth\nScience\nClimate\nRace\n\n\n\n\nCulture\nExpand/collapse submenu for Culture\n\n\nBooks\nMovies\nTelevision\nPop Culture\nFood\nArt & Design \nPerforming Arts\nLife Kit\n\n\n\n\nMusic\nExpand/collapse submenu for Music\n\n\n\n        Tiny Desk\n    \n\n\n        #NowPlaying\n    \n\n\n        All Songs Considered\n    \n\n\n        Music Features\n    \n\n\n        Live Sessions\n    \n\n\n\n\nPodcasts & Shows\nExpand/collapse submenu for Podcasts & Shows\n\n\nDaily\n\n\n\n\n                    

There's some tricky Javascript still nesting within it, but it definitely cleaned up a bit. On other websites, you may find even clearer text extraction.

As detailed in the [BeautifulSoup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), the easiest way to navigate through the tags is to simply name the tag you're interested in. For example:

In [10]:
soup.head # fetches the head tag, which ecompasses the title tag

<head><!-- OneTrust Cookies Consent Notice start for npr.org -->
<script src="https://cdn.cookielaw.org/consent/82089dfe-410c-4e1b-a7f9-698174b62a86/OtAutoBlock.js" type="text/javascript"></script>
<script charset="UTF-8" data-domain-script="82089dfe-410c-4e1b-a7f9-698174b62a86" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript"></script>
<script type="text/javascript">
function OptanonWrapper() {
    NPR_OptanonWrapper = true;
    document.dispatchEvent(new CustomEvent('npr:DataConsentAvailable'));
    
    OneTrust.OnConsentChanged(function() {
        document.dispatchEvent(new CustomEvent('npr:DataConsentChanged'));
    });
 }
</script>
<!-- OneTrust Cookies Consent Notice end for npr.org -->
<script ccpa-opt-out-geo="us" ccpa-opt-out-ids="C0004" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript"></script><script class="optanon-category-C0004" type="text/plain">
    document.cookie =

Usually head tags are small and only contain the most important contents; however, here, there's some Javascript code. The ``title`` tag resides within the head tag.

In [11]:
soup.title # we can specifically call for the title tag

<title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title>

This result includes the tag itself. To get just the text within the tags, we can use the ``.name`` property.

In [12]:
soup.title.string

'What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR'

We can navigate to the parent tag (the tag that encompasses the current tag) via the ``.parent`` attribute:

In [13]:
soup.title.parent.name

'head'

# 3. Parse the page with Beautiful Soup
In HTML code, paragraphs are often denoated with a ``<p>`` tag.

In [14]:
soup.p

<p class="byline__name byline__name--block">
<a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
      Domenico Montanaro
    </a>
</p>

This returns the first paragraph, and we can access properties of the given tag with the same syntax we use for dictionaries and dataframes:

In [15]:
soup.p['class']

['byline__name', 'byline__name--block']

In addition to 'paragraph' (aka p) tags, link tags are also very common and are denoted by ``<a>`` tags

In [16]:
soup.a

<a class="skiplink" href="#mainContent">Skip to main content</a>

It is called the a tag because links are also called 'anchors'. Nearly every page has multiple paragraphs and anchors, so how do we access the subsequent tags? There are two common functions, `.find()` and `.find_all()`.

In [17]:
soup.find('title')

<title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title>

In [18]:
soup.find_all('title')

[<title>What If The Polls Are Wrong Again? 4 Scenarios For What Might Happen In The Elections : NPR</title>]

Here, the results were seemingly the same, since there is only one title to a webpage. However, you'll notice that ``.find_all()`` returned a list, not a single item. Sure, there was only one item in the list, but it returned a list. As the name implies, find_all() returns all items that match the passed-in tag.

In [19]:
soup.find_all('a')

[<a class="skiplink" href="#mainContent">Skip to main content</a>,
 <a class="skiplink" href="https://help.npr.org/contact/s/article?name=what-are-the-keyboard-shortcuts-for-using-the-npr-org-audio-player">Keyboard shortcuts for audio player</a>,
 <a data-metrics-action="click npr logo" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr logo","clickType":"global navigation","clickUrl":"\/"}' href="/">
 <img alt="NPR logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
 </a>,
 <a data-metrics-action="click npr shop" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr shop","clickType":"global navigation","clickUrl":"https://shopnpr.org"}' href="https://shopnpr.org">NPR Shop</a>,
 <a data-metrics-action="click npr logo" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr logo","clickType":"global navigation","clickUrl":"\/"}' href="/">Home</a>,
 <a data-metrics-action="click news" data-metrics-ga4='{"action":"global_navigat

Look at all of those links! Amazing. It might be hard to read but the **href** portion of an *a* tag denotes the URL, and we can capture it via the ``.get()`` function.

In [20]:
#for link in soup.find_all('a'): # we could optionally pass the href=True flag .find_all('a', href=True)
    #print(link.get('href'))
 
soup.find_all('a', href = True)

[<a class="skiplink" href="#mainContent">Skip to main content</a>,
 <a class="skiplink" href="https://help.npr.org/contact/s/article?name=what-are-the-keyboard-shortcuts-for-using-the-npr-org-audio-player">Keyboard shortcuts for audio player</a>,
 <a data-metrics-action="click npr logo" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr logo","clickType":"global navigation","clickUrl":"\/"}' href="/">
 <img alt="NPR logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
 </a>,
 <a data-metrics-action="click npr shop" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr shop","clickType":"global navigation","clickUrl":"https://shopnpr.org"}' href="https://shopnpr.org">NPR Shop</a>,
 <a data-metrics-action="click npr logo" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr logo","clickType":"global navigation","clickUrl":"\/"}' href="/">Home</a>,
 <a data-metrics-action="click news" data-metrics-ga4='{"action":"global_navigat

Many of those links are relative to the current URL (e.g., /section/news/).

In [21]:
paragraphs = soup.find_all('p')
paragraphs

[<p class="byline__name byline__name--block">
 <a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
       Domenico Montanaro
     </a>
 </p>,
 <p>
                 Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.
                 <b aria-label="Image credit" class="credit">
                     
                     Scott Olson/Getty Images
                     
                 </b>
 <b class="hide-caption"><b>hide caption</b></b>
 </p>,
 <p class="caption">Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.</p>,
 <p>There's a lot that can happen Tuesday, the culmination of a long midterm election campaign that will provide the first nationwide measure of the U.S. electorate since Donald Trump was elected president.</p>,
 <p>One narrative has become

If we want just the paragraph text:

In [22]:
for pa in paragraphs:
    print(pa.get_text())



      Domenico Montanaro
    


                Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.
                
                    
                    Scott Olson/Getty Images
                    
                
hide caption

Supporters of Missouri Democratic Sen. Claire McCaskill wait for her to arrive at a campaign stop in St. Louis on Monday.
There's a lot that can happen Tuesday, the culmination of a long midterm election campaign that will provide the first nationwide measure of the U.S. electorate since Donald Trump was elected president.
One narrative has become dominant: that Democrats are likely to gain control of the House and Republicans hold the Senate, if not expand their majority there. That narrative is based largely on national polls, and caution should be urged. Pollsters have made a lot of adjustments to hopefully correct what they got wrong in 2016, but they can't tell you precisely who 

Since there are multiple tags and various attributes, it is useful to check the data type of BeautifulSoup objects:

In [23]:
type(soup.find('p'))

bs4.element.Tag

Since the ``.find()`` function returns a BeautifulSoup element, we can tack on multiple calls that continue to return elements:

In [24]:
soup.find('p')

<p class="byline__name byline__name--block">
<a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
      Domenico Montanaro
    </a>
</p>

In [25]:
#soup.find('p').find('a')
soup.p.a

<a data-metrics='{"action":"Click Byline","category":"Story Metadata"}' href="https://www.npr.org/people/392602474/domenico-montanaro" rel="author">
      Domenico Montanaro
    </a>

In [26]:
#soup.find('p').find('a').attrs['href'] # att
soup.p.a['href']

'https://www.npr.org/people/392602474/domenico-montanaro'

In [27]:
#soup.find('p').find('a').text
(soup.p.a.text).strip()

'Domenico Montanaro'

That doesn't look pretty, but it makes sense because if you look at what ``.find('a')`` returned, there is plenty of whitespace. We can remove that with Python's built-in ``.strip()`` function.

In [28]:
soup.find('p').find('a').text.strip()

'Domenico Montanaro'

**NOTE:** above, we accessed the attributes of a link by using the property ``.attrs``. ``.attrs`` takes a dictionary as a parameter, and in the example above, we only provided the _key_, not a _value_, too. That is, we only cared that the ``<a>`` tag had an attribute named ``href`` (which we grabbed by typing that command), and we made no specific demands on what the value must be. In other words, regardless of the value of _href_, we grabbed that element. Alternatively, if you inspect your HTML code and notice select regions for which you'd like to extract text, you can specify it as part of the attributes, too!

For example, in the full ``response.text``, we see the following line:

``<header class="npr-header" id="globalheader" aria-label="NPR header">``

Let's say that we know that the information we care about is within tags that match this template (i.e., **class** is an attribute, and its value is **'npr-header'**).

In [29]:
#soup.find('header', attrs={'class':'npr-header'})
#soup.find('header', class_ = 'npr-header')
soup.select('header.npr-header')

[<header aria-label="NPR header" class="npr-header" id="globalheader">
 <nav class="navigation">
 <div class="navigation__top-level-wrap" data-metrics-all="" data-metrics-category="global navigation" data-metrics-category-ga4="recirculation">
 <ul class="navigation__top-level">
 <li class="navigation__toggle-wrap">
 <button aria-expanded="false" aria-haspopup="true" class="navigation__toggle navigation__toggle--open" data-metrics-action="open navigation menu" data-metrics-ga4='{"action":"global_navigation_click","clickName":"open navigation menu","clickType":"global navigation"}' id="navigation__toggle--open">
                     Open Navigation Menu
                 </button>
 </li>
 <li class="navigation__home">
 <a data-metrics-action="click npr logo" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr logo","clickType":"global navigation","clickUrl":"\/"}' href="/">
 <img alt="NPR logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
 </a>
 </li>
 <li id="

This matched it! We could then continue further processing by tacking on other commands:

In [30]:
soup.find('header', attrs={'class':'npr-header'}).find_all("li") # li stands for list items
soup.select('header.npr-header')[0].find_all('li')

[<li class="navigation__toggle-wrap">
 <button aria-expanded="false" aria-haspopup="true" class="navigation__toggle navigation__toggle--open" data-metrics-action="open navigation menu" data-metrics-ga4='{"action":"global_navigation_click","clickName":"open navigation menu","clickType":"global navigation"}' id="navigation__toggle--open">
                     Open Navigation Menu
                 </button>
 </li>,
 <li class="navigation__home">
 <a data-metrics-action="click npr logo" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr logo","clickType":"global navigation","clickUrl":"\/"}' href="/">
 <img alt="NPR logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
 </a>
 </li>,
 <li id="navigation__station-mount"></li>,
 <li class="navigation__account" id="user-account-dropdown-mount"></li>,
 <li class="navigation__shop">
 <a data-metrics-action="click npr shop" data-metrics-ga4='{"action":"global_navigation_click","clickName":"npr shop","clickType":"global n

This returns all of our list items, and since it's within a particular header section of the page, it appears they are links to menu items for navigating the webpage. If we wanted to grab just the links within these:

In [31]:
menu_links = set()
for list_item in soup.find('header', attrs={'class':'npr-header'}).find_all("li"):
    for link in list_item.find_all('a', href=True):
        menu_links.add(link)
menu_links # a unique set of all the seemingly important links in the header

{<a data-metrics-action="click #nowplaying" data-metrics-ga4='{"category":"recirculation","action":"global_navigation_click","clickName":"#nowplaying","clickType":"music ecosystem navigation","clickUrl":"https:\/\/www.npr.org\/series\/1002643619\/now-playing"}' href="https://www.npr.org/series/1002643619/now-playing">
         #NowPlaying
     </a>,
 <a data-metrics-action="click about npr" data-metrics-ga4='{"action":"global_navigation_click","clickName":"about npr","clickType":"about ecosystem navigation","clickUrl":"/about/"}' href="/about/">About NPR</a>,
 <a data-metrics-action="click all songs considered" data-metrics-ga4='{"category":"recirculation","action":"global_navigation_click","clickName":"all songs considered","clickType":"music ecosystem navigation","clickUrl":"https:\/\/www.npr.org\/sections\/allsongs\/"}' href="https://www.npr.org/sections/allsongs/">
         All Songs Considered
     </a>,
 <a data-metrics-action="click all things considered" data-metrics-ga4='{"act

## TAKEAWAY LESSON
The above tutorial isn't meant to be a study guide to memorize; its point is to show you the most important functionaity that exist within BeautifulSoup, and to illustrate how one can access different pieces of content. No two web scraping tasks are identical, so it's useful to play around with code and try different things, while using the above as examples of how you may navigate between different tags and properties of a page. Don't worry; we are always here to help when you get stuck!

# String formatting
As we parse webpages, we may often want to further adjust and format the text to a certain way.

For example, say we wanted to scrape a polical website that lists all US Senators' name and office phone number. We may want to store information for each senator in a dictionary. All senators' information may be stored in a list. Thus, we'd have a list of dictionaries. Below, we will initialize such a list of dictionary (it has only 3 senators, for illustrative purposes, but imagine it contains many more).

In [32]:
# this is a bit clumsy of an initialization, but we spell it out this way for clarity purposes
# NOTE: imagine the dictionary were constructed in a more organic manner
senator1 = {"name":"Lamar Alexander", "number":"555-229-2812"}
senator2 = {"name":"Tammy Baldwin", "number":"555-922-8393"}
senator3 = {"name":"John Barrasso", "number":"555-827-2281"}
senators = [senator1, senator2, senator3]
print(senators)

[{'name': 'Lamar Alexander', 'number': '555-229-2812'}, {'name': 'Tammy Baldwin', 'number': '555-922-8393'}, {'name': 'John Barrasso', 'number': '555-827-2281'}]


In the real-world, we may not want the final form of our information to be in a Python dictionary; rather, we may need to send an email to people in our mailing list, urging them to call their senators. If we have a templated format in mind, we can do the following:

In [33]:
email_template = """Please call {name} at {number}"""
for senator in senators:
    print(email_template.format(**senator))

Please call Lamar Alexander at 555-229-2812
Please call Tammy Baldwin at 555-922-8393
Please call John Barrasso at 555-827-2281


**Please [visit here](https://docs.python.org/3/library/stdtypes.html#str.format)** for further documentation
                      
Alternatively, one can also format their text via the ``f'-strings`` property. [See documentation here](https://docs.python.org/3/reference/lexical_analysis.html#f-strings). For example, using the above data structure and goal, one could yield identical results via:

In [34]:
for senator in senators:
    print(f"Please call {senator['name']} at {senator['number']}")

Please call Lamar Alexander at 555-229-2812
Please call Tammy Baldwin at 555-922-8393
Please call John Barrasso at 555-827-2281


Additionally, sometimes we wish to search large strings of text. If we wish to find all occurrences within a given string, a very mechanical, procedural way of doing it would be to use the ``.find()`` function in Python and to repeatedly update the starting index from which we are looking.

## Regular Expressions
A way more suitable and powerful way is to use Regular Expressions, which is a pattern matching mechanism used throughout Computer Science and programming (it's not just specific to Python). A tutorial on Regular Expressions (aka regex) is beond this lab, but below are many great resources that we recommend, if you are interested in them (could be very useful for a homework problem):
- https://docs.python.org/3.3/library/re.html
- https://regexone.com
- https://docs.python.org/3/howto/regex.html.

# Additonal Python/Homework Comment
In Homework #1, we ask you to complete functions that have signatures with a syntax you may not have seen before:

``def create_star_table(starlist: list) -> list:``

To be clear, this syntax merely means that the input parameter must be a list, and the output must be a list. It's no different than any other function, it just puts a requirement on the behavior of the function.

It is **typing** our function. Please [see this documention if you have more questions.](https://docs.python.org/3/library/typing.html)

# Walkthrough Example (of Web Scraping)
We're going to see the structure of Goodread's best books list (**NOTE: Goodreads is described a little more within the other Lab2_More_Pandas.ipynb notebook)**. We'll use the Developer tools in chrome, safari and firefox have similar tools available. To get this page we use the `requests` module. But first we should check if the company's policy allows scraping. Check the [robots.txt](https://www.goodreads.com/robots.txt) to find what sites/elements are not accessible. Please read and verify.

![](images/goodreads1.png)

In [35]:
url="https://www.npr.org/2018/11/05/664395755/what-if-the-polls-are-wrong-again-4-scenarios-for-what-might-happen-in-the-elect"
response = requests.get(url)
# response.status_code
# response.content

# Beautiful Soup (library) time!
soup = BeautifulSoup(response.content, "html.parser")
    #print(soup)
    # soup.prettify()
soup.find("title")

    # Q1: how do we get the title's text?
soup.title.text
    # Q2: how do we get the webpage's entire content?
soup.prettify()

'<!DOCTYPE html>\n<html class="no-js" lang="en">\n <head>\n  <!-- OneTrust Cookies Consent Notice start for npr.org -->\n  <script src="https://cdn.cookielaw.org/consent/82089dfe-410c-4e1b-a7f9-698174b62a86/OtAutoBlock.js" type="text/javascript">\n  </script>\n  <script charset="UTF-8" data-domain-script="82089dfe-410c-4e1b-a7f9-698174b62a86" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript">\n  </script>\n  <script type="text/javascript">\n   function OptanonWrapper() {\n    NPR_OptanonWrapper = true;\n    document.dispatchEvent(new CustomEvent(\'npr:DataConsentAvailable\'));\n    \n    OneTrust.OnConsentChanged(function() {\n        document.dispatchEvent(new CustomEvent(\'npr:DataConsentChanged\'));\n    });\n }\n  </script>\n  <!-- OneTrust Cookies Consent Notice end for npr.org -->\n  <script ccpa-opt-out-geo="us" ccpa-opt-out-ids="C0004" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/java

In [36]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
url = URLSTART+BESTBOOKS+'1'
print(url)
page = requests.get(url)

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1


We can see properties of the page. Most relevant are `status_code` and `text`. The former tells us  if the web-page was found, and if found , ok. (See lecture notes.)

In [37]:
page.status_code # 200 is good


504

In [38]:
page.text[:5000]

'<!DOCTYPE html>\n<html>\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <title>Goodreads request took too long</title>\n    <style type="text/css">\n        * {\n            margin: 0;\n        }\n        html, body {\n            height: 100%;\n            font-family: georgia, serif;\n        }\n        .wrapper {\n            min-height: 100%;\n            height: auto !important;\n            height: 100%;\n            margin: 0 auto -100px; /* the bottom margin is the negative value of the footer\'s height */\n        }\n        .tfooter, .push {\n            height: 100px; /* .push must be the same height as .footer */\n        }\n            /* end sticky footer */\n\n        * {\n            margin: 0;\n        }\n\n        html, body {\n            height: 100%;\n        }\n\n        .wrapper {\n            min-height: 100%;\n            height: auto !important;\n            height: 100%;\n            margin: 0 auto -100px;\n        }\n\n

Let us write a loop to fetch 2 pages of "best-books" from goodreads. Notice the use of a format string. This is an example of old-style python format strings

In [39]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
for i in range(1,3):
    bookpage=str(i)
    stuff=requests.get(URLSTART+BESTBOOKS+bookpage)
    #print(URLSTART+BESTBOOKS+bookpage)
    filetowrite= "pages"+ '%02d' % i + ".html"
    print("FTW", filetowrite)
    fd=open(filetowrite,"w", encoding="utf-8")
    fd.write(stuff.text)
    fd.close()
    time.sleep(2)

FTW pages01.html
FTW pages02.html


## 2. Parse the page, extract book urls

Notice how we do file input-output, and use beautiful soup in the code below. The `with` construct ensures that the file being read is closed, something we do explicitly for the file being written. We look for the elements with class `bookTitle`, extract the urls, and write them into a file

In [40]:
bookdict={}
for i in range(1,3):
    books=[]
    stri = '%02d' % i
    filetoread="pages"+ stri + '.html'
    print("FTW", filetoread)
    with open(filetoread) as fdr:
        data = fdr.read()
    soup = BeautifulSoup(data, 'html.parser')
    for e in soup.select('.bookTitle'):
        books.append(e['href'])
    print(books[:10])
    bookdict[stri]=books
    print(bookdict)
    fd=open("list"+stri+'.txt',"w",encoding="utf-8" )
    fd.write("\n".join(books))
    fd.close()

FTW pages01.html
['/book/show/2767052-the-hunger-games', '/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix', '/book/show/1885.Pride_and_Prejudice', '/book/show/2657.To_Kill_a_Mockingbird', '/book/show/19063.The_Book_Thief', '/book/show/41865.Twilight', '/book/show/170448.Animal_Farm', '/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set', '/book/show/11127.The_Chronicles_of_Narnia', '/book/show/11870085-the-fault-in-our-stars']
{'01': ['/book/show/2767052-the-hunger-games', '/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix', '/book/show/1885.Pride_and_Prejudice', '/book/show/2657.To_Kill_a_Mockingbird', '/book/show/19063.The_Book_Thief', '/book/show/41865.Twilight', '/book/show/170448.Animal_Farm', '/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set', '/book/show/11127.The_Chronicles_of_Narnia', '/book/show/11870085-the-fault-in-our-stars', '/book/show/18405.Gone_with_the_Wind', '/book/show/370493.The_Giving_Tree', '/book/show/5297.The_Picture_of_Dorian_Gray', '/book/show/6185.Wuthering

Here is George Orwell's 1984

In [41]:
print(bookdict['02'][0])
bookdict['01'][0]

/book/show/662.Atlas_Shrugged


'/book/show/2767052-the-hunger-games'

 Lets go look at the first URLs on both pages

![](images/goodreads2.png)

## 3. Parse a book page, extract book properties

Ok so now lets dive in and get one of these these files and parse them.

In [42]:
furl=URLSTART+bookdict['02'][0]
furl

'https://www.goodreads.com/book/show/662.Atlas_Shrugged'

![](images/goodreads3.png)

In [43]:
fstuff=requests.get(furl)
#fstuff = requests.get('https://www.goodreads.com/book/show/43763.Interview_with_the_Vampire')
print(fstuff.status_code)

200


In [53]:
#d=BeautifulSoup(fstuff.text, 'html.parser')
# try this to take care of arabic strings
d = BeautifulSoup(fstuff.text, 'html.parser', from_encoding="utf-8")


C:\Users\Admin\anaconda3\lib\site-packages\bs4\__init__.py:226: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


<!DOCTYPE html>
<html lang="en"><head><script>var ue_t0=window.ue_t0||+new Date();(function(e){var c=e,a={main_scope:"mainscopecsm",q:[],t0:c.ue_t0||+new Date(),d:g};function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(k,j,h){var i={m:k,f:j,l:h,fromOnError:1,args:arguments};c.ueLogError(i);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(function(e,d){var a=e.ue||{};function c(g){if(!g){return}var f=d.head||d.getElementsByTagName("head")[0]||d.documentElement,h=d.createElement("script");h.async="async";h.src=g;f.insertBefore(h,f.firstChild)}function b(){var k=e.ue_cdn||"z-ecx.images-amazon.com",g=e.ue_cdns||"images-na.ssl-images-amazon.com",j=

In [250]:
import json
d.select("meta[property='og:title']")[0]['content']
data = d.select("script[type='application/ld+json']")[0].string
json_data = json.loads(data)
isbn = json_data['isbn']
#authors = [author['name'] for author in json_data['author']]
#authors = ' and '.join(authors)
#author = d.find_all('div', class_='ContributorLinksList')[0].find_all('a', class_='ContributorLink')
ratingCount = json_data['aggregateRating']['ratingCount']
reviewCount = json_data['aggregateRating']['reviewCount']
author_links = d.select('.ContributorLinksList a.ContributorLink')
author = [link['href'] for link in author_links]
year = d.select('.FeaturedDetails p')[1]
year = year.text[-4:] 


Lets get everything we want...

In [241]:
#d=BeautifulSoup(fstuff.text, 'html.parser', from_encoding="utf-8")
print(
"title:" , d.select("meta[property='og:title']")[0]['content'],"\n",
"isbn:", isbn ,"\n",
"type:", d.select("meta[property='og:type']")[0]['content'],"\n",
"author:", author,"\n",
"year:", year,"\n",
#"average rating", d.select_one("span.average").text,"\n",
"ratingCount:", ratingCount ,"\n"
#reviewCount:", reviewCount
)

title: Atlas Shrugged 
 isbn: 9780452011878 
 type: books.book 
 author: ['https://www.goodreads.com/author/show/432.Ayn_Rand', 'https://www.goodreads.com/author/show/1421.Leonard_Peikoff'] 
 year: 1957 
 ratingCount: 380854 



Ok, now that we know what to do, lets wrap our fetching into a proper script. So that we dont overwhelm their servers, we will only fetch 5 from each page, but you get the idea...

We'll segue of a bit to explore new style format strings. See https://pyformat.info for more info.

In [47]:
"list{:0>2}.txt".format(3)

'list03.txt'

In [48]:
a = "4"
b = 4
class Four:
    def __str__(self):
        return "Nineteen"
c=Four()

In [49]:
"The hazy cat jumped over the {} and {} and {}".format(a, b, c)

'The hazy cat jumped over the 4 and 4 and Nineteen'

## 4. Set up a pipeline for fetching and parsing

Ok lets get back to the fetching...

In [68]:
import os
fetched=[]
for i in range(1,3):
    with open("list{:0>2}.txt".format(i)) as fd:
        counter=0
        for bookurl_line in fd:
            if counter > 4:
                break
            bookurl=bookurl_line.strip()
            html = URLSTART+bookurl
            stuff=requests.get(html)
            filetowrite=bookurl.split('/')[-1]
            os.makedirs("files/", exist_ok=True)
            filetowrite1="files/"+str(i)+"_"+filetowrite+".html"
            print("FTW", filetowrite1)
            fd=open(filetowrite1,"w", encoding = "utf-8")
            fd.write(stuff.text)
            fd.close()
            fetched.append(filetowrite1)
            time.sleep(2)
            counter=counter+1
            #with open(filetowrite, "w", encoding="utf-8") as fw:
                #fw.write(stuff.text)
            #fetched.append(filetowrite)
            #time.sleep(2)
            #counter = counter + 1
            
            
print(fetched)

FTW files/1_2767052-the-hunger-games.html
FTW files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html
FTW files/1_1885.Pride_and_Prejudice.html
FTW files/1_2657.To_Kill_a_Mockingbird.html
FTW files/1_19063.The_Book_Thief.html
FTW files/2_662.Atlas_Shrugged.html
FTW files/2_33648131-the-notebook.html
FTW files/2_6.Harry_Potter_and_the_Goblet_of_Fire.html
FTW files/2_99107.Winnie_the_Pooh.html
FTW files/2_4381.Fahrenheit_451.html
['files/1_2767052-the-hunger-games.html', 'files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html', 'files/1_1885.Pride_and_Prejudice.html', 'files/1_2657.To_Kill_a_Mockingbird.html', 'files/1_19063.The_Book_Thief.html', 'files/2_662.Atlas_Shrugged.html', 'files/2_33648131-the-notebook.html', 'files/2_6.Harry_Potter_and_the_Goblet_of_Fire.html', 'files/2_99107.Winnie_the_Pooh.html', 'files/2_4381.Fahrenheit_451.html']


Ok we are off to parse each one of the html pages we fetched. We have provided the skeleton of the code and the code to parse the year, since it is a bit more complex...see the difference in the screenshots above. 

In [281]:
#import re
#yearre = r'\d{4}'
#def get_year(d):
    #if d.select_one("nobr.greyText"):
        #return d.select_one("nobr.greyText").text.strip().split()[-1][:-1]
    #else:
        #thetext=d.select("div#details div.row")[1].text.strip()
        #rowmatch=re.findall(yearre, thetext)
        #if len(rowmatch) > 0:
            #rowtext=rowmatch[0].strip()
        #else:
            #rowtext="NA"
        #return rowtext

#get_year(d)


[]

<div class="exercise"><b>Exercise</b></div>

Your job is to fill in the code to get the genres.

In [305]:
#def get_genres(d):
    # your code here
    #genres=d.select("div.elementList div.left a")
    #glist=[]
    #for g in genres:
        #glist.append(g['href'])
    #return glist

#get_genres(d)

data = d.select("script[type='application/json']")[0].string
print(data)
json_data = json.loads(data)
#genres = [bg['name'] for bg in json_data['bookGenres']]
#genres
#authors = ' and '.join(authors)
#def get_genres(d):
    #glist = []
    #genres=d.select("script
    #for lnk in genres :
        #glist.append(lnk.text)
    #return print(glist)
#get_genres(d)

{"props":{"pageProps":{"apolloState":{"ROOT_QUERY":{"__typename":"Query","getAdsTargeting({\"getAdsTargetingInput\":{\"contextual\":{\"legacyId\":\"662\",\"legacyResourceType\":\"book\"}}})":{"__typename":"AdsTargeting","behavioral":null,"contextual":{"__typename":"ContextualTargeting","resource":"Work_817219","tags":["1","24","128","384","25","144","28","2207","985","587"],"adult":false}},"getSiteHeaderBanner":{"__typename":"SiteHeaderBanner","altText":"Discover the Best Books of Summer 2023","clickthroughUrl":"https://www.goodreads.com/blog/show/2573?ref=TopSummerBooks23_eb","desktop1xPhoto":"https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/siteheaderbannerimages/1682972303i/249.jpg","desktop2xPhoto":"https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/siteheaderbannerimages/1682972309i/250.jpg","mobile1xPhoto":"https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/siteheaderbannerimages/1682972314i/251.jpg","mobile2xPhoto":"https://i.gr-assets.co

In [ ]:

listofdicts=[]
for filetoread in fetched:
    print(filetoread)
    td={}
    with open(filetoread) as fd:
        datext = fd.read()
    d=BeautifulSoup(datext, 'html.parser')
    td['title']=d.select_one("meta[property='og:title']")['content']
    td['isbn']=d.select_one("meta[property='books:isbn']")['content']
    td['booktype']=d.select_one("meta[property='og:type']")['content']
    td['author']=d.select_one("meta[property='books:author']")['content']
    td['rating']=d.select_one("span.average").text
    td['year'] = get_year(d)
    td['file']=filetoread
    glist = get_genres(d)
    td['genres']="|".join(glist)
    listofdicts.append(td)


In [ ]:
listofdicts[0]

Finally lets write all this stuff into a csv file which we will use to do analysis.

In [ ]:
df = pd.DataFrame.from_records(listofdicts)
df

In [ ]:
df.to_csv("files/meta_utf8_EK.csv", index=False, header=True)